In [1]:
from shutil import copy2, copystat, Error, ignore_patterns
import os
from PIL import Image

import numpy as np
def copytree(src, dst, symlinks=False, copy_function=copy2,
             ignore_dangling_symlinks=False):
    """Recursively copy a directory tree.

    The destination directory must not already exist.
    If exception(s) occur, an Error is raised with a list of reasons.

    If the optional symlinks flag is true, symbolic links in the
    source tree result in symbolic links in the destination tree; if
    it is false, the contents of the files pointed to by symbolic
    links are copied. If the file pointed by the symlink doesn't
    exist, an exception will be added in the list of errors raised in
    an Error exception at the end of the copy process.

    You can set the optional ignore_dangling_symlinks flag to true if you
    want to silence this exception. Notice that this has no effect on
    platforms that don't support os.symlink.

    The optional ignore argument is a callable. If given, it
    is called with the `src` parameter, which is the directory
    being visited by copytree(), and `names` which is the list of
    `src` contents, as returned by os.listdir():

        callable(src, names) -> ignored_names

    Since copytree() is called recursively, the callable will be
    called once for each directory that is copied. It returns a
    list of names relative to the `src` directory that should
    not be copied.

    The optional copy_function argument is a callable that will be used
    to copy each file. It will be called with the source path and the
    destination path as arguments. By default, copy2() is used, but any
    function that supports the same signature (like copy()) can be used.

    """
    ignore=ignore_patterns('*.tiff')
    names = os.listdir(src)
    if ignore is not None:
        ignored_names = ignore(src, names)
    else:
        ignored_names = set()

    os.makedirs(dst)
    errors = []
    for name in names:
        srcname = os.path.join(src, name)
        if name in ignored_names:
            # im = Image.open(srcname)
            im_original = Image.open(srcname)
            # im_original.show()
            im_original = np.array(im_original)
            orig_min = 0
            orig_max = im_original.max()
            target_min = 0.0
            target_max = 255.0
            im_original_scaled = (np.array(im_original)-orig_min)*((target_max-
            target_min)/(orig_max-orig_min))+target_min
            mammogram_uint8_by_function = im_original_scaled.astype(np.uint8)
            # print(orig_max)
            im = Image.fromarray(np.array(mammogram_uint8_by_function, dtype='uint8'))
            # image_pil = Image.fromarray(im, 'I;16')
            dstname = os.path.join(dst, name.replace('.tiff',f'_max_val{orig_max}.jpeg'))

            # np.save(dstname,im)
            # im.show()
            im.save(dstname)

            # cv2.imwrite(dstname, im)

            continue
        else:
            dstname = os.path.join(dst, name)

        try:
            if os.path.islink(srcname):
                linkto = os.readlink(srcname)
                if symlinks:
                    # We can't just leave it to `copy_function` because legacy
                    # code with a custom `copy_function` may rely on copytree
                    # doing the right thing.
                    os.symlink(linkto, dstname)
                    copystat(srcname, dstname, follow_symlinks=not symlinks)
                else:
                    # ignore dangling symlink if the flag is on
                    if not os.path.exists(linkto) and ignore_dangling_symlinks:
                        continue
                    # otherwise let the copy occurs. copy2 will raise an error
                    if os.path.isdir(srcname):
                        copytree(srcname, dstname, symlinks,
                                 copy_function)
                    else:
                        copy_function(srcname, dstname)
            elif os.path.isdir(srcname):
                copytree(srcname, dstname, symlinks, copy_function)
            else:
                # Will raise a SpecialFileError for unsupported file types
                copy_function(srcname, dstname)
        # catch the Error from the recursive copytree so that we can
        # continue with other files
        except Error as err:
            errors.extend(err.args[0])
        except OSError as why:
            errors.append((srcname, dstname, str(why)))
    try:
        copystat(src, dst)
    except OSError as why:
        # Copying file access times may fail on Windows
        if getattr(why, 'winerror', None) is None:
            errors.append((src, dst, str(why)))
    if errors:
        raise Error(errors)
    return dst

src='/home/roironen/GSD/GSD/data/GSD1A-LIVEEXP20R2P1__2022-08-23T12_21_31-Measurement 1'
dst='/home/roironen/GSD/GSD/data/GSD1A-LIVEEXP20R2P1__2022-08-23T12_21_31-Measurement 1_jpeg'
copytree(src, dst)


'/home/roironen/GSD/GSD/data/uri-gsd1a-liveexp19r2p2__2022-07-19T16_18_38-Measurement_1_jpeg'